In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
# Helper funtion to show an image with cv2
def cv_show_img(title, image, wait=0):
    cv2.namedWindow(title)
    cv2.startWindowThread()
    cv2.imshow(title, image)
    cv2.waitKey(wait)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

# Helper funtion to show multiple images at the same time
def cv_show_mult_img(titleArr, imageArr, wait=0):
    for i in range(len(titleArr)):
        cv2.namedWindow(titleArr[i])
        cv2.startWindowThread()
        cv2.imshow(titleArr[i], imageArr[i])
    cv2.waitKey(wait)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [3]:
oxf = cv2.imread('./images/oxford.jpg')

## Blurring & Sharpening
- Helps computer vision algorithms work much better because they're very sensitive to edges
- Blurring is averaging pixels over kernels in convolution
- Sharpening is mphasizing the center points in kernels during convolution

In [5]:
# create a 3x3 & 7x7 kernel to convolve over the image
kernel3 = np.ones((3,3), np.uint8) / (3*3)
kernel7 = np.ones((7,7), np.uint8) / (7*7)

# Use cv2.filter2D to convolve over the image
blurred3 = cv2.filter2D(oxf,-1,kernel3) # just always use -1 for image depth
blurred7 = cv2.filter2D(oxf,-1,kernel7)

cv_show_mult_img(['Original','3x3 Kernel Blur','7x7 Kernel Blur'],[oxf,blurred3,blurred7])

#### Different Blurring Methods
- Gaussian - Blur based on the gaussian mean of the kernel
- Median - take the median of all pixels in the kernel and replace the center pixel with that value
- Bilateral - effective at noise removal
- Average (default)

In [6]:
blur_avg = cv2.blur(oxf, (3,3))
blur_gaus = cv2.GaussianBlur(oxf, (3,3), 0) # the last 0 defines the std dev of the Gaussian function used in blurring
blur_med = cv2.medianBlur(oxf, 3)
blur_b = cv2.bilateralFilter(oxf, 9, 75, 75) # 9 is just 3 * 3, the 75's are idk but should be the same

cv_show_mult_img(
    ['Original','Average Blur','Gaussian Blur','Median Blur','Bilateral Blur'],
    [oxf,blur_avg,blur_gaus,blur_med,blur_b]
)

## Sharpening

In [7]:
# Not super useful idk
sharp_filter = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
sharpened = cv2.filter2D(oxf,-1,sharp_filter)

cv_show_mult_img(['Original','Sharpened'],[oxf,sharpened])

## Thresholding & Binarization
- Binarization is changing the pixel values from 0-255 to 0-1 (Typically changed to either 0 or 255 based on some logic)
- Thresholding is the act of converting an image to binary form based on some logic
- Thresholding requires grayscale images

#### Adaptive Thresholding - use an algo to figure out the best thresholding for your image

In [10]:
img = cv2.imread('./images/gradient.jpg',0)

In [11]:
# Thresholding Examples

# 127 and up goes to black, rest goes to white
ret, thresh1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
# Reverse
ret, thresh2 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

# Values above 127 get sent to 127, rest stay at their values (grayscale)
ret, thresh3 = cv2.threshold(img, 127, 255, cv2.THRESH_TRUNC)
# Values below 127 go to 0, rest are unchanged
ret, thresh4 = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO)
# Reverse
ret, thresh5 = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO_INV)

cv_show_mult_img(
    ['Original','1. Binary','2. Inverse Binary','3. Truncate','4. To Zero','5. Inverse To Zero'],
    [img, thresh1, thresh2, thresh3, thresh4, thresh5]
)

Biggest downfall of thresholding is you need to provide the 127 value before running these functions

We can use Adaptive thresholding to threshold more intelligently

In [13]:
darwin = cv2.imread('./images/Origin_of_Species.jpg', 0)        # the 0 second arg loads the image as grayscale
cv_show_img('Darwin',darwin)

In [18]:
# good practice to blur to remove noise in the image
darwin = cv2.GaussianBlur(darwin, (3,3), 0)

# Lets see a binary threshold to start
ret, thresh_bin = cv2.threshold(darwin, 127, 255, cv2.THRESH_BINARY)

# Thresholoding with adaptive threshold
thresh_ada = cv2.adaptiveThreshold(darwin, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 3, 5)

# Otsu's algorithm for thresholding
_, thresh_otsu = cv2.threshold(darwin, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv_show_mult_img(
    ['Original', 'Binary Threshold','Adaptive Thresholding',"Otsu's Thresholding"],
    [darwin, thresh_bin, thresh_ada, thresh_otsu]
)

## Dilation & Erosion
Dilation adds pixels to the boundaries of objects in an image, while erosion removes pixels from the boundaries of an image
Reminder: these work better on black backgrounds with white objects... they reverse direction on white backgrounds with black objects

In [19]:
cv = cv2.imread('./images/opencv_inv.png')
cv_show_img('Open CV', cv)

In [20]:
# Define a Kernel
kernel = np.ones((5,5), np.uint8)

erosion = cv2.erode(cv, kernel, iterations=1)
dilation = cv2.dilate(cv, kernel, iterations=1)

cv_show_mult_img(
    ['Original','Eroded','Dilated'],
    [cv,erosion,dilation]
)

## Edge Detection & Image Gradients

In [25]:
oxf = cv2.imread('./images/oxford.jpg', 0)
height, width = oxf.shape[:2]

# Sobel Edges
sobel_x = cv2.Sobel(oxf, cv2.CV_64F, 0, 1, ksize=5)
sobel_y = cv2.Sobel(oxf, cv2.CV_64F, 1, 0, ksize=5)
sobel_or = cv2.bitwise_or(sobel_x, sobel_y)

# LaPlacian Edges
laplacian = cv2.Laplacian(oxf, cv2.CV_64F)

cv_show_mult_img(
    ['Original','Sobel X','Sobel Y','Sobel OR','LaPlacian'],
    [oxf,sobel_x,sobel_y,sobel_or,laplacian]
)

Canny Edge Detection Works Best

In [26]:
# Canny Edge Detection uses gradient values as thresholds
canny1 = cv2.Canny(oxf, 50, 120)
canny2 = cv2.Canny(oxf, 10, 200)
canny3 = cv2.Canny(oxf, 200, 240)
canny4 = cv2.Canny(oxf, 70, 110)

cv_show_mult_img(
    ['Original','Canny Original','Canny Wide','Canny High','Canny Narrow'],
    [oxf,canny1,canny2,canny3,canny4]
)